In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
dataset = pd.read_csv("Data.csv").drop(["Margin", "Bat","Toss", "BR", "Ground"], axis =1)

In [ ]:
dataset.columns

Index(['Team', 'Result', 'Opposition', 'Start Date'], dtype='object')

In [ ]:
dataset["Team"].unique()

array(['West Indies', 'South Africa', 'Australia', 'India', 'Sri Lanka',
       'England', 'Afghanistan', 'Pakistan', 'Bangladesh', 'New Zealand'],
      dtype=object)

In [ ]:
dataset["Opposition"].replace({"v\xa0South Africa": "South Africa", 'v\xa0West Indies':"West Indies", 'v\xa0Australia':"Australia",
       'v\xa0Sri Lanka':"Sri Lanka", 'v\xa0England':"England", 'v\xa0Pakistan':"Pakistan",
       'v\xa0Afghanistan':"Afghanistan", 'v\xa0Bangladesh':"Bangladesh", 'v\xa0New Zealand':"New Zealand",
       'v\xa0India':"India"}, inplace=True)

In [ ]:
dataset['Start Date'] = pd.to_datetime(dataset['Start Date'])
dataset = dataset[~(dataset['Start Date'] < '2017-04-18')].reset_index(drop = True)

In [ ]:
dataset.tail

<bound method NDFrame.tail of              Team Result    Opposition Start Date
0         England    won     v Ireland 2017-05-05
1         England    won     v Ireland 2017-05-07
2     New Zealand    won     v Ireland 2017-05-14
3     New Zealand    won    Bangladesh 2017-05-17
4      Bangladesh   lost   New Zealand 2017-05-17
..            ...    ...           ...        ...
546    Bangladesh    won    v Zimbabwe 2020-03-06
547  South Africa    won     Australia 2020-03-07
548     Australia   lost  South Africa 2020-03-07
549     Australia    won   New Zealand 2020-03-13
550   New Zealand   lost     Australia 2020-03-13

[551 rows x 4 columns]>

In [ ]:
grouped_data = dataset.groupby("Team")

In [ ]:
grouped_data.get_group("India")

,Team,Result,Opposition,Start Date
19,India,won,Pakistan,2017-06-04
26,India,lost,Sri Lanka,2017-06-08
33,India,won,South Africa,2017-06-11
41,India,won,Bangladesh,2017-06-15
44,India,lost,Pakistan,2017-06-18
...,...,...,...,...
520,India,won,Australia,2020-01-17
522,India,won,Australia,2020-01-19
527,India,lost,New Zealand,2020-02-05
529,India,lost,New Zealand,2020-02-08


In [ ]:
Teams = ['West Indies', 'South Africa', 'Australia', 'England', 'Sri Lanka',
       'Pakistan', 'Bangladesh', 'Afghanistan', 'India', 'New Zealand']
sequences = {}
for team in Teams:
    data = grouped_data.get_group(team)
    idxs = data.index
    s = ""
    for i in idxs:
        if(data.Result[i]=="won"):
            s+="W"
        else:
            s+="N"
    sequences[team] = s


In [ ]:
sequences

{'Afghanistan': 'WNWNNWNWWWNNNWWWWWNWWWNNNWNWNWNWNNNNNNNNNNNN',
 'Australia': 'NNNNWNNNNWNNNNNNNWNWNNNNWWWWWWWWWWNWWWWWNNWNNNNNW',
 'Bangladesh': 'NWWNWNNNNWWWNNWNWWNNWWNWWWWNWNNNWWWWWNNWNWNNNNNWWW',
 'England': 'WWWWNWWWNWWWWWWWNWNWWNWNWWWWWNWWWWWNWNWNWWWWWWNWWWNNWWWNNW',
 'India': 'WNWWNWWNWWWWWWWWWNWNWWNWWWWWNWWWNNWWWWNWWNNWWNWWWWWNWWWNNNWWWWWNWWNWWNWWNWWNNN',
 'New Zealand': 'WWWNNNWNNWWWWWWWWWNNWNWNWWWNNNWNWWWWWWWWNNNWNWWWN',
 'Pakistan': 'NWWWWWWWWWNNNNNWWWWWWNWNNNWWNNWNNNNNNNNNNNWNNWWWWWW',
 'South Africa': 'NNWWNNWWWNNNWNNWWWNNWWWWNWNWWNWWWWWWNNNWNNWWWNWWW',
 'Sri Lanka': 'NWNNWWNNNNNNNNNNNNWNNNNWWWNNNWWNNNNNWNNNNNNNNWNWNWNWNWWWNNWWW',
 'West Indies': 'NWNNWNNNNNNNNWWWNWWNNWNNNWNNNWNNWNWWNWNNWNNNNNNWNNWWWWNNWWWNNN'}

In [ ]:
def extract_state_changes(s):
    dic = {
        "WW": 0,
        "WN": 0,
        "NW": 0,
        "NN": 0,
    }
    n = len(s)
    for i in range(1,n,1):
        if(s[i-1]=="W" and s[i]=="W"):
            dic["WW"]+=1
        elif(s[i-1]=="W" and s[i]=="N"):
            dic["WN"]+=1
        elif(s[i-1]=="N" and s[i]=="W"):
            dic["NW"]+=1
        elif(s[i-1]=="N" and s[i]=="N"):
            dic["NN"]+=1
        else:
            print("Something unexpected happened at index: ", i)
    return dic


In [ ]:
state = {}
for team in Teams:
    state[team] = extract_state_changes(sequences[team])

In [ ]:
state

{'Afghanistan': {'NN': 16, 'NW': 9, 'WN': 10, 'WW': 8},
 'Australia': {'NN': 20, 'NW': 8, 'WN': 7, 'WW': 13},
 'Bangladesh': {'NN': 12, 'NW': 12, 'WN': 11, 'WW': 14},
 'England': {'NN': 2, 'NW': 13, 'WN': 13, 'WW': 29},
 'India': {'NN': 6, 'NW': 17, 'WN': 18, 'WW': 36},
 'New Zealand': {'NN': 8, 'NW': 9, 'WN': 10, 'WW': 21},
 'Pakistan': {'NN': 18, 'NW': 7, 'WN': 6, 'WW': 19},
 'South Africa': {'NN': 9, 'NW': 11, 'WN': 10, 'WW': 18},
 'Sri Lanka': {'NN': 29, 'NW': 12, 'WN': 11, 'WW': 8},
 'West Indies': {'NN': 24, 'NW': 14, 'WN': 14, 'WW': 9}}

In [ ]:
def extract_state_changes1(s):
    dic = {
        "WW": 0,
        "WN": 0,
        "NW": 0,
        "NN": 0,
    }
    n = len(s)
    for i in range(1,n,1):
        if(s[i-1]=="W" and s[i]=="W"):
            dic["WW"]+=1
        elif(s[i-1]=="W" and s[i]=="N"):
            dic["WN"]+=1
        elif(s[i-1]=="N" and s[i]=="W"):
            dic["NW"]+=1
        elif(s[i-1]=="N" and s[i]=="N"):
            dic["NN"]+=1
        else:
            print("Something unexpected happened at index: ", i)
    arr = np.array([[dic["WW"],dic["WN"]],[dic["NW"],dic["NN"]]])
    return arr


In [ ]:
transition_mat = {}
for team in Teams:
    transition_mat[team] = extract_state_changes1(sequences[team])

In [ ]:
transition_mat["India"]

array([[36, 18],
       [17,  6]])

In [ ]:
def extract_state_changes2(s):
    dic = {
        "WW": 0,
        "WN": 0,
        "NW": 0,
        "NN": 0,
    }
    n = len(s)
    for i in range(1,n,1):
        if(s[i-1]=="W" and s[i]=="W"):
            dic["WW"]+=1
        elif(s[i-1]=="W" and s[i]=="N"):
            dic["WN"]+=1
        elif(s[i-1]=="N" and s[i]=="W"):
            dic["NW"]+=1
        elif(s[i-1]=="N" and s[i]=="N"):
            dic["NN"]+=1
        else:
            print("Something unexpected happened at index: ", i)
    a = 0
    b = 0
    c = 0
    d = 0
    if((dic["WW"]+dic["WN"])!=0):
        a = dic["WW"]/(dic["WW"]+dic["WN"])
        b = dic["WN"]/(dic["WW"]+dic["WN"])
    if((dic["NW"]+dic["NN"])!=0):
        c = dic["NW"]/(dic["NW"]+dic["NN"])
        d = dic["NN"]/(dic["NW"]+dic["NN"])
    arr = np.array([[a,b],[c,d]])
    return arr


In [ ]:
transition_mat_f = {}
for team in Teams:
    transition_mat_f[team] = extract_state_changes2(sequences[team])

In [ ]:
x = transition_mat_f["Afghanistan"]

In [ ]:
x

array([[0.44444444, 0.55555556],
       [0.36      , 0.64      ]])

In [ ]:
alpha_w = x[1][0]/(1+x[1][0]-x[0][0])
alpha_n = (1-x[0][0])/(1+x[1][0]-x[0][0])
print(alpha_w)
print(alpha_n)
print(alpha_w+alpha_n)

0.3932038834951457
0.6067961165048544
1.0


In [ ]:
t = []
Wining_Prob = []
Loosing_Prob = []
WW = []
WN = []
NW = []
NN = []
for team in Teams:
    x = transition_mat_f[team]
    t.append(team)
    Wining_Prob.append(x[1][0]/(1+x[1][0]-x[0][0]))
    Loosing_Prob.append((1-x[0][0])/(1+x[1][0]-x[0][0]))
    WW.append(x[0][0])
    WN.append(x[0][1])
    NW.append(x[1][0])
    NN.append(x[1][1])
final_result = pd.DataFrame({
    "Team": t,
    "WW": WW,
    "WN": WN,
    "NW": NW,
    "NN": NN,
    "Wining Probablity": Wining_Prob,
    "Loosing Probablity": Loosing_Prob,
}).sort_values("Wining Probablity", ascending=False).reset_index(drop = True)


In [ ]:
final_result

,Team,WW,WN,NW,NN,Wining Probablity,Loosing Probablity
0,England,0.690476,0.309524,0.866667,0.133333,0.736842,0.263158
1,India,0.666667,0.333333,0.739130,0.260870,0.689189,0.310811
2,New Zealand,0.677419,0.322581,0.529412,0.470588,0.621381,0.378619
3,South Africa,0.642857,0.357143,0.550000,0.450000,0.606299,0.393701
4,Pakistan,0.760000,0.240000,0.280000,0.720000,0.538462,0.461538
5,Bangladesh,0.560000,0.440000,0.500000,0.500000,0.531915,0.468085
6,Australia,0.650000,0.350000,0.285714,0.714286,0.449438,0.550562
7,Afghanistan,0.444444,0.555556,0.360000,0.640000,0.393204,0.606796
8,West Indies,0.391304,0.608696,0.368421,0.631579,0.377049,0.622951
9,Sri Lanka,0.421053,0.578947,0.292683,0.707317,0.335788,0.664212


Head-On Analysis

In [ ]:
grouped_data_2 = dataset.groupby(["Team","Opposition"])

In [ ]:
base_team = "India"          #Change base_team to get head on results for different teams
# data = grouped_data_2.get_group((base_team, "England"))

In [ ]:
data

,Team,Result,Opposition,Start Date
465,Australia,won,New Zealand,2019-06-29
549,Australia,won,New Zealand,2020-03-13


In [ ]:
sequences = {}
for team in Teams:
    if(team == base_team): continue
    data = grouped_data_2.get_group((base_team,team))
    idxs = data.index
    s = ""
    for i in idxs:
        if(data.Result[i]=="won"):
            s+="W"
        else:
            s+="N"
    sequences[team] = s

In [ ]:
sequences

{'Afghanistan': 'NW',
 'Australia': 'WWWNWNWWWWNNNWNWW',
 'Bangladesh': 'WWWW',
 'England': 'WNNN',
 'New Zealand': 'NWWWWWNWNNNN',
 'Pakistan': 'WNWWW',
 'South Africa': 'WWWWNWWW',
 'Sri Lanka': 'NWWWWWNWWW',
 'West Indies': 'WWNWWNNWWWWWNWW'}

In [ ]:
state = {}
for team in Teams:
    if team == base_team: continue
    state[team] = extract_state_changes(sequences[team])
state

{'Afghanistan': {'NN': 0, 'NW': 1, 'WN': 0, 'WW': 0},
 'Australia': {'NN': 2, 'NW': 4, 'WN': 4, 'WW': 6},
 'Bangladesh': {'NN': 0, 'NW': 0, 'WN': 0, 'WW': 3},
 'England': {'NN': 2, 'NW': 0, 'WN': 1, 'WW': 0},
 'New Zealand': {'NN': 3, 'NW': 2, 'WN': 2, 'WW': 4},
 'Pakistan': {'NN': 0, 'NW': 1, 'WN': 1, 'WW': 2},
 'South Africa': {'NN': 0, 'NW': 1, 'WN': 1, 'WW': 5},
 'Sri Lanka': {'NN': 0, 'NW': 2, 'WN': 1, 'WW': 6},
 'West Indies': {'NN': 1, 'NW': 3, 'WN': 3, 'WW': 7}}

In [ ]:
transition_mat = {}
for team in Teams:
    if team == base_team: continue
    transition_mat[team] = extract_state_changes1(sequences[team])
transition_mat["Afghanistan"]

array([[0, 0],
       [1, 0]])

In [ ]:
transition_mat_f = {}
for team in Teams:
    if team == base_team: continue
    transition_mat_f[team] = extract_state_changes2(sequences[team])
transition_mat_f["Bangladesh"]

array([[1., 0.],
       [0., 0.]])

In [ ]:
t = []
Wining_Prob = []
Loosing_Prob = []
for team in Teams:
    if team == base_team: continue
    x = transition_mat_f[team]
    t.append(team)
    Wining_Prob.append(x[1][0]/(1+x[1][0]-x[0][0]))
    Loosing_Prob.append((1-x[0][0])/(1+x[1][0]-x[0][0]))
final_result = pd.DataFrame({
    "Against": t,
    "Wining Probablity": Wining_Prob,
    "Loosing Probablity": Loosing_Prob,
}).sort_values("Wining Probablity", ascending=False).reset_index(drop = True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


In [ ]:
final_result

,Against,Wining Probablity,Loosing Probablity
0,Sri Lanka,0.875000,0.125000
1,South Africa,0.857143,0.142857
2,Pakistan,0.750000,0.250000
3,West Indies,0.714286,0.285714
4,Australia,0.625000,0.375000
5,New Zealand,0.545455,0.454545
6,Afghanistan,0.500000,0.500000
7,England,0.000000,1.000000
8,Bangladesh,NaN,NaN
